In [0]:
import datetime
users = [
    {
        "id": 1,
        "first_name": "Corrie",
        "last_name": "Van den Oord",
        "email": "cvandenoord@etsy.com",
        "phone_numbers": {"mobile": "+0 000 000 000", "home": "+0 111 111 111"},
        "is_customer": True,
        "amount_paid": 1000.55,
        "customer_from": datetime.date(2021,1,15),
        "last_updated_ts": datetime.datetime(2021, 2, 10, 1, 15, 0)
    },
    {
        "id": 2,
        "first_name": "Nikolaus",
        "last_name": "Brewitt",
        "email": "brewitt@etsy.com",
        "phone_numbers": {"mobile": "+1 000 000 000", "home": "+1 111 111 111"},
        "is_customer": True,
        "amount_paid": 900.55,
        "customer_from": datetime.date(2021,2,15),
        "last_updated_ts": datetime.datetime(2021, 2, 17, 1, 15, 0)
    },
    {
        "id": 3,
        "first_name": "Nikolaus",
        "last_name": "Brewitt",
        "email": "brewitt@etsy.com",
        "phone_numbers": None,
        "is_customer": True,
        "amount_paid": 900.55,
        "customer_from": datetime.date(2021,2,15),
        "last_updated_ts": datetime.datetime(2021, 2, 17, 1, 15, 0)
    }
]


In [0]:
from pyspark.sql import Row

In [0]:
users_df = spark.createDataFrame([Row(**user) for user in users])

In [0]:
users_df.printSchema()

root
-- id: long (nullable = true)
-- first_name: string (nullable = true)
-- last_name: string (nullable = true)
-- email: string (nullable = true)
-- phone_numbers: map (nullable = true)
 |-- key: string
 |-- value: string (valueContainsNull = true)
-- is_customer: boolean (nullable = true)
-- amount_paid: double (nullable = true)
-- customer_from: date (nullable = true)
-- last_updated_ts: timestamp (nullable = true)

In [0]:
users_df.show()

+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+
 id|first_name| last_name| email| phone_numbers|is_customer|amount_paid|customer_from| last_updated_ts|
+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+
 1| Corrie|Van den Oord|cvandenoord@etsy.com|{mobile -> +0 000...| true| 1000.55| 2021-01-15|2021-02-10 01:15:00|
 2| Nikolaus| Brewitt| brewitt@etsy.com|{mobile -> +1 000...| true| 900.55| 2021-02-15|2021-02-17 01:15:00|
 3| Nikolaus| Brewitt| brewitt@etsy.com| null| true| 900.55| 2021-02-15|2021-02-17 01:15:00|
+---+----------+------------+--------------------+--------------------+-----------+-----------+-------------+-------------------+

In [0]:
users_df.select("id", "phone_numbers").show(truncate=False)

+---+--------------------------------------------------+
id |phone_numbers |
+---+--------------------------------------------------+
1 |{mobile -> +0 000 000 000, home -> +0 111 111 111}|
2 |{mobile -> +1 000 000 000, home -> +1 111 111 111}|
3 |null |
+---+--------------------------------------------------+

In [0]:
from pyspark.sql.functions import col

In [0]:
users_df.select("id", col("phone_numbers")["mobile"]).show()

+---+---------------------+
 id|phone_numbers[mobile]|
+---+---------------------+
 1| +0 000 000 000|
 2| +1 000 000 000|
 3| null|
+---+---------------------+

In [0]:
users_df.select("id", col("phone_numbers")["mobile"].alias("mobile")).show()

+---+--------------+
 id| mobile|
+---+--------------+
 1|+0 000 000 000|
 2|+1 000 000 000|
 3| null|
+---+--------------+

In [0]:
users_df.select("id", col("phone_numbers")["mobile"].alias("mobile"), col("phone_numbers")["home"].alias("home")).show()

+---+--------------+--------------+
 id| mobile| home|
+---+--------------+--------------+
 1|+0 000 000 000|+0 111 111 111|
 2|+1 000 000 000|+1 111 111 111|
 3| null| null|
+---+--------------+--------------+

In [0]:
from pyspark.sql.functions import explode

In [0]:
users_df.select("id", explode("phone_numbers")).show()

+---+------+--------------+
 id| key| value|
+---+------+--------------+
 1|mobile|+0 000 000 000|
 1| home|+0 111 111 111|
 2|mobile|+1 000 000 000|
 2| home|+1 111 111 111|
+---+------+--------------+

In [0]:
from pyspark.sql.functions import explode_outer

In [0]:
users_df.select("id", explode_outer("phone_numbers")).show()

+---+------+--------------+
 id| key| value|
+---+------+--------------+
 1|mobile|+0 000 000 000|
 1| home|+0 111 111 111|
 2|mobile|+1 000 000 000|
 2| home|+1 111 111 111|
 3| null| null|
+---+------+--------------+

In [0]:
users_df.select("*", explode("phone_numbers")). \
    withColumnRenamed("key", "phone_type"). \
    withColumnRenamed("value", "phone_number").\
    drop("phone_numbers"). \
    show()

+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+----------+--------------+
 id|first_name| last_name| email|is_customer|amount_paid|customer_from| last_updated_ts|phone_type| phone_number|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+----------+--------------+
 1| Corrie|Van den Oord|cvandenoord@etsy.com| true| 1000.55| 2021-01-15|2021-02-10 01:15:00| mobile|+0 000 000 000|
 1| Corrie|Van den Oord|cvandenoord@etsy.com| true| 1000.55| 2021-01-15|2021-02-10 01:15:00| home|+0 111 111 111|
 2| Nikolaus| Brewitt| brewitt@etsy.com| true| 900.55| 2021-02-15|2021-02-17 01:15:00| mobile|+1 000 000 000|
 2| Nikolaus| Brewitt| brewitt@etsy.com| true| 900.55| 2021-02-15|2021-02-17 01:15:00| home|+1 111 111 111|
+---+----------+------------+--------------------+-----------+-----------+-------------+-------------------+----------+--------------+